In [ ]:
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
import os
import numpy as np
import pandas as pd

# テキストファイルをnumpy配列に読み込み
all_data = np.loadtxt("claim_toiawase_data_python.txt")
#numpy配列をDataFrame型に変換(でーたを見やすくするため)
all_data_pd = pd.DataFrame(all_data)
#DataFrame型の列名を設定
all_data_pd.columns = ["seq", "Imp", "Imp_val", "m_sum_neg", "c_neg", "m_sum_all", "m_sum_nzr", "Mag", "c_nzr", "c_all", "m_max", "s_min", "m_sdv_neg", 
                   "s_sdv_neg", "m_avg_neg", "s_avg_neg", "s_sdv_nzr", "m_sdv_all", "s_sdv_all", "s_avg_nzr", "m_avg_nzr", "s_avg_all", "Sc",
                   "m_avg_all", "m_sum_pos", "c_pos", "m_min", "s_max", "m_sdv_pos", "s_sdv_pos", "m_avg_pos", "s_avg_pos", "m_sdv_nzr"]
print(all_data_pd)   #ImpとImp_valは同じ

# 特徴量の部分のみall_Xに読み込み
all_X = all_data[:9813,3:33]
#ラベル部分をall_yに読み込み
all_y = all_data[:9813, 2]
print(all_X)
print(all_y)
# print(type(all_X))


         seq  Imp  Imp_val  m_sum_neg  c_neg  m_sum_all  m_sum_nzr  Mag  \
0        1.0  1.0      1.0        1.0    3.0        1.1        1.1  1.2   
1        2.0  1.0      1.0        0.5    1.0        0.5        0.5  0.5   
2        3.0  1.0      1.0        2.1    3.0        2.1        2.1  2.2   
3        4.0  1.0      1.0        1.9    3.0        1.9        1.9  2.0   
4        5.0  1.0      1.0        0.4    1.0        0.5        0.5  0.5   
5        6.0  1.0      1.0        0.5    1.0        0.5        0.5  0.5   
6        7.0  1.0      1.0        3.0    6.0        3.0        3.0  3.2   
7        8.0  1.0      1.0        1.7    3.0        2.6        2.6  2.8   
8        9.0  1.0      1.0        1.1    3.0        1.3        1.3  1.5   
9       10.0  1.0      1.0        1.3    2.0        1.3        1.3  1.3   
10      11.0  1.0      1.0        3.8    5.0        3.8        3.8  4.1   
11      12.0  1.0      1.0        1.0    2.0        1.0        1.0  1.0   
12      13.0  1.0      1.

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

all_X = all_X[::, :18]

for c in [0.1,1.0,10.0,100.0,1000.0,10000.0,100000.0]:
    for g in [1.0,0.1,0.01,0.001,0.0001,0.00001]:
        
#       十回分のそれぞれのスコアを格納する空のlistを作成
        accuracy_train = []
        f1_train = []
        precision_train = []
        recall_train = []
        
        accuracy_test = []
        f1_test = []
        precision_test = []
        recall_test = []
#       パイプラインの作成
        pipe = make_pipeline(StandardScaler(), SVC(C=c, gamma=g, kernel="linear", random_state=1))
        #層化10-fold交差検証の開始
        kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=2)
        for train_index, test_index in kf.split(all_X, all_y):
            train_X, test_X = all_X[train_index], all_X[test_index]
            train_y, test_y = all_y[train_index], all_y[test_index]
#             訓練データのラベル1のsmote前のデータ数の出力
#             print("train_X(label=1): %s" %train_X[train_y == 1].shape[0])
#             print("train_X(label=0): %s" %train_X[train_y == 0].shape[0])
            smote = SMOTE("minority", random_state=2)
#             train_X_smote , train_y_smote = smote.fit_sample(train_X, train_y)
#             print("smote_train_X(label=1): %s" %train_X_smote[ train_y_smote ==1 ].shape[0])
#             print("smote_train_X(label=0): %s" %train_X_smote[ train_y_smote ==0 ].shape[0])
            enn = EditedNearestNeighbours("all",random_state=2)
#             train_X_enn , train_y_enn = enn.fit_sample(train_X, train_y)
#             print("enn_train_X(label=1): %s" %train_X_enn[ train_y_enn ==1 ].shape[0])
#             print("enn_train_X(label=0): %s" %train_X_enn[ train_y_enn ==0 ].shape[0])
            #SMOTEENNを行い新しい訓練データを作成
            train_X_smoteenn , train_y_smoteenn = SMOTEENN(random_state=2, smote=smote,enn=enn ).fit_sample(train_X, train_y)
            print("smoteenn_train_X(label=1): %s" %train_X_smoteenn[ train_y_smoteenn ==1 ].shape[0])
            print("smoteenn_train_X(label=0): %s" %train_X_smoteenn[ train_y_smoteenn ==0 ].shape[0])
            pipe.fit(train_X_smoteenn, train_y_smoteenn)
            
            pre_label_train = pipe.predict(train_X_smoteenn)
            accuracy_train.append(accuracy_score(train_y_smoteenn, pre_label_train))
            f1_train.append(f1_score(train_y_smoteenn, pre_label_train))
            precision_train.append(precision_score(train_y_smoteenn, pre_label_train))
            recall_train.append(recall_score(train_y_smoteenn, pre_label_train))
            
            pre_label_test = pipe.predict(test_X)
            accuracy_test.append(accuracy_score(test_y, pre_label_test))
            f1_test.append(f1_score(test_y, pre_label_test))
            precision_test.append(precision_score(test_y, pre_label_test))
            recall_test.append(recall_score(test_y, pre_label_test))
        
        #十回のスコアの平均と標準偏差の出力
        print("\n")
        print("C:%s      gamma:%s" %(c,g))
        print("accuracy_train: %.4f +- %.4f" %(np.mean(accuracy_train), np.std(accuracy_train)))
        print("f1_train: %.4f +- %.4f" %(np.mean(f1_train), np.std(f1_train)))
        print("precision_train: %.4f +- %.4f" %(np.mean(precision_train), np.std(precision_train)))
        print("recall_train: %.4f +- %.4f" %(np.mean(recall_train), np.std(recall_train)))
        
        print("accuracy_test: %.4f +- %.4f" %(np.mean(accuracy_test), np.std(accuracy_test)))
        print("f1_test: %.4f +- %.4f" %(np.mean(f1_test), np.std(f1_test)))
        print("precision_test: %.4f +- %.4f" %(np.mean(precision_test), np.std(precision_test)))
        print("recall_test: %.4f +- %.4f" %(np.mean(recall_test), np.std(recall_test)))
        print("list_shape: %s" %len(accuracy_train))
        print("\n")
    

smoteenn_train_X(label=1): 4559
smoteenn_train_X(label=0): 5005
smoteenn_train_X(label=1): 4502
smoteenn_train_X(label=0): 5107
smoteenn_train_X(label=1): 4512
smoteenn_train_X(label=0): 5321
smoteenn_train_X(label=1): 4606
smoteenn_train_X(label=0): 4798
smoteenn_train_X(label=1): 4598
smoteenn_train_X(label=0): 5321
smoteenn_train_X(label=1): 4576
smoteenn_train_X(label=0): 5720
smoteenn_train_X(label=1): 4549
smoteenn_train_X(label=0): 5228
smoteenn_train_X(label=1): 4592
smoteenn_train_X(label=0): 4440
smoteenn_train_X(label=1): 4608
smoteenn_train_X(label=0): 5297
smoteenn_train_X(label=1): 4561
smoteenn_train_X(label=0): 5415


C:0.1      gamma:1.0
accuracy_train: 0.9075 +- 0.0055
f1_train: 0.9016 +- 0.0039
precision_train: 0.9008 +- 0.0076
recall_train: 0.9027 +- 0.0103
accuracy_test: 0.8015 +- 0.0099
f1_test: 0.4866 +- 0.0189
precision_test: 0.3885 +- 0.0171
recall_test: 0.6518 +- 0.0318
list_shape: 10


smoteenn_train_X(label=1): 4559
smoteenn_train_X(label=0): 5005
smoteenn_t

smoteenn_train_X(label=1): 4559
smoteenn_train_X(label=0): 5005
smoteenn_train_X(label=1): 4502
smoteenn_train_X(label=0): 5107
smoteenn_train_X(label=1): 4512
smoteenn_train_X(label=0): 5321
smoteenn_train_X(label=1): 4606
smoteenn_train_X(label=0): 4798
smoteenn_train_X(label=1): 4598
smoteenn_train_X(label=0): 5321
smoteenn_train_X(label=1): 4576
smoteenn_train_X(label=0): 5720
smoteenn_train_X(label=1): 4549
smoteenn_train_X(label=0): 5228
smoteenn_train_X(label=1): 4592
smoteenn_train_X(label=0): 4440
smoteenn_train_X(label=1): 4608
smoteenn_train_X(label=0): 5297
smoteenn_train_X(label=1): 4561
smoteenn_train_X(label=0): 5415


C:1.0      gamma:0.001
accuracy_train: 0.9075 +- 0.0064
f1_train: 0.9014 +- 0.0044
precision_train: 0.9037 +- 0.0076
recall_train: 0.8992 +- 0.0107
accuracy_test: 0.8034 +- 0.0096
f1_test: 0.4892 +- 0.0204
precision_test: 0.3917 +- 0.0174
recall_test: 0.6525 +- 0.0349
list_shape: 10


smoteenn_train_X(label=1): 4559
smoteenn_train_X(label=0): 5005
smoteenn

smoteenn_train_X(label=1): 4559
smoteenn_train_X(label=0): 5005
smoteenn_train_X(label=1): 4502
smoteenn_train_X(label=0): 5107
smoteenn_train_X(label=1): 4512
smoteenn_train_X(label=0): 5321
smoteenn_train_X(label=1): 4606
smoteenn_train_X(label=0): 4798
smoteenn_train_X(label=1): 4598
smoteenn_train_X(label=0): 5321
smoteenn_train_X(label=1): 4576
smoteenn_train_X(label=0): 5720
smoteenn_train_X(label=1): 4549
smoteenn_train_X(label=0): 5228
smoteenn_train_X(label=1): 4592
smoteenn_train_X(label=0): 4440
smoteenn_train_X(label=1): 4608
smoteenn_train_X(label=0): 5297
smoteenn_train_X(label=1): 4561
smoteenn_train_X(label=0): 5415


C:100.0      gamma:1.0
accuracy_train: 0.9080 +- 0.0068
f1_train: 0.9018 +- 0.0046
precision_train: 0.9050 +- 0.0075
recall_train: 0.8988 +- 0.0090
accuracy_test: 0.8039 +- 0.0084
f1_test: 0.4884 +- 0.0210
precision_test: 0.3918 +- 0.0164
recall_test: 0.6490 +- 0.0364
list_shape: 10


smoteenn_train_X(label=1): 4559
smoteenn_train_X(label=0): 5005
smoteenn

smoteenn_train_X(label=1): 4559
smoteenn_train_X(label=0): 5005
smoteenn_train_X(label=1): 4502
smoteenn_train_X(label=0): 5107
smoteenn_train_X(label=1): 4512
smoteenn_train_X(label=0): 5321
smoteenn_train_X(label=1): 4606
smoteenn_train_X(label=0): 4798
smoteenn_train_X(label=1): 4598
smoteenn_train_X(label=0): 5321
smoteenn_train_X(label=1): 4576
smoteenn_train_X(label=0): 5720
smoteenn_train_X(label=1): 4549
smoteenn_train_X(label=0): 5228
smoteenn_train_X(label=1): 4592
smoteenn_train_X(label=0): 4440
smoteenn_train_X(label=1): 4608
smoteenn_train_X(label=0): 5297
smoteenn_train_X(label=1): 4561
smoteenn_train_X(label=0): 5415


C:1000.0      gamma:0.001
accuracy_train: 0.9081 +- 0.0066
f1_train: 0.9020 +- 0.0045
precision_train: 0.9049 +- 0.0071
recall_train: 0.8992 +- 0.0092
accuracy_test: 0.8039 +- 0.0083
f1_test: 0.4887 +- 0.0199
precision_test: 0.3920 +- 0.0157
recall_test: 0.6497 +- 0.0364
list_shape: 10


smoteenn_train_X(label=1): 4559
smoteenn_train_X(label=0): 5005
smote

In [ ]:
a = np.array(accuracy_train)
print(np.mean(accuracy_train))
print(a.mean())


b = [1,1,1,1,1,1,1,0,0,0,0,0,1,0,1,0,1]
c = [0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0]
print(confusion_matrix(b,c,labels=[1,0]))
print(recall_score(b,c))
print(len(b))

print(all_X)
print(all_X[:, :9].shape)